In [9]:
from pathlib import Path
from pydub import AudioSegment
from utils import read_rttm

idx = 0
min_length_seconds = 0.1

audio_path = Path(f"data/audio_16khz_mono_wav/{idx}.wav")
diarization_path = Path(f"data/diarization/{idx}.rttm")
diarization_df = read_rttm(diarization_path)



tempdir = Path("./temp_files")
tempdir.mkdir(exist_ok=True)

audio = AudioSegment.from_wav(audio_path)
filenames = []
for i, row in diarization_df.iterrows():
    filename = audio_path.with_suffix("").name + f"__{row['start']}__{row['end']}.wav"
    start = int(1000*row["start"])
    end = int(1000*row["end"])
    outpath = str(tempdir/filename)
    filenames.append(outpath)
    audio[start:end].export(outpath, format="wav")
diarization_df["audio"] = filenames

for i in tempdir.glob("*.wav"):
    i.unlink()
tempdir.rmdir()


In [7]:
diarization_df

,start,end,speaker_name
0,3.862,7.988,SPEAKER_08
1,8.531,9.346,SPEAKER_08
2,10.671,17.360,SPEAKER_01
3,21.944,24.762,SPEAKER_08
4,24.932,25.866,SPEAKER_08
...,...,...,...
1093,2980.484,2983.370,SPEAKER_02
1094,2983.998,2984.168,SPEAKER_02
1095,2984.796,2984.949,SPEAKER_02
1096,2985.000,2985.017,SPEAKER_02


In [5]:
tempdir.rmdir

AttributeError: 'NoneType' object has no attribute 'rmdir'